In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
from moveit_msgs.srv import GetStateValidity, GetStateValidityRequest
from moveit_msgs.srv import GetJointWithConstraints, GetJointWithConstraintsRequest
from moveit_msgs.msg import RobotState
from moveit_msgs.msg import Constraints, OrientationConstraint
from moveit_msgs.msg import MoveItErrorCodes
from geometry_msgs.msg import Quaternion, Pose, PoseStamped, Point32
import trimesh
from trimesh import transformations
import numpy as np
from sensor_msgs.msg import PointCloud2, PointField, PointCloud
import struct

In [ ]:
def quaternion_to_rotation_matrix(q):
    # Normalise the quaternion to unit length
    q = q / np.linalg.norm(q)
    
    # Extract the values
    qw, qx, qy, qz = q[0], q[1], q[2], q[3]

    # Create the rotation matrix
    rotation_matrix = np.array([
        [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw, 0.0],
        [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw, 0.0],
        [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2, 0.0],
        [0.0,0.0,0.0,1.0]
    ])
    
    return rotation_matrix

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)

In [ ]:
rospy.init_node('constrained_motion_planning_test', anonymous=True)

In [ ]:
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
move_group = moveit_commander.MoveGroupCommander("arm")
joint_names = move_group.get_active_joints()
state_validity_service = rospy.ServiceProxy('/check_state_validity', GetStateValidity)
sample_joint_with_constraints_service = rospy.ServiceProxy('/sample_joint_with_constraints', GetJointWithConstraints)

In [ ]:
# load the obstacle
shelf_pose = PoseStamped()
shelf_pose.header.frame_id = "base_link"
shelf_pose.pose.position.x = 0.88
shelf_pose.pose.position.y = 0.23
shelf_pose.pose.position.z = 0
shelf_pose.pose.orientation.x = 0
shelf_pose.pose.orientation.y = 0
shelf_pose.pose.orientation.z = 1
shelf_pose.pose.orientation.w = 0
scene.add_mesh('shelf', shelf_pose, 'constraint_scene/Gruppe_21/model.stl', (0.02, 0.03, 0.02))

In [ ]:
def sample_points_on_mesh(mesh, num_points):
    # Calculate the area of each face
    face_areas = mesh.area_faces

    # Normalize the face areas to create a probability distribution
    face_probs = face_areas / face_areas.sum()

    # Sample face indices based on their probabilities
    sampled_face_indices = np.random.choice(len(mesh.faces), size=num_points, p=face_probs)

    # Sample barycentric coordinates for each point
    u = np.random.rand(num_points, 1)
    v = np.random.rand(num_points, 1)
    out_of_range = u + v > 1
    u[out_of_range] = 1 - u[out_of_range]
    v[out_of_range] = 1 - v[out_of_range]
    w = 1 - u - v

    # Calculate the 3D Cartesian coordinates of the sampled points
    vertices = mesh.vertices[mesh.faces[sampled_face_indices]]
    sampled_points = u * vertices[:, 0] + v * vertices[:, 1] + w * vertices[:, 2]

    return sampled_points

In [ ]:
def numpy_to_pointcloud2(points, frame_id="base_link"):
    '''
    convert pointcloud from numpy format to PointCloud2 in the base_link frame.
    '''
    pc2_msg = PointCloud2()
    pc2_msg.header.stamp = rospy.Time.now()
    pc2_msg.header.frame_id = frame_id
    pc2_msg.height = 1
    pc2_msg.width = len(points)
    pc2_msg.fields = [
        PointField(name="x", offset=0, datatype=PointField.FLOAT32, count=1),
        PointField(name="y", offset=4, datatype=PointField.FLOAT32, count=1),
        PointField(name="z", offset=8, datatype=PointField.FLOAT32, count=1),
    ]
    pc2_msg.is_bigendian = False
    pc2_msg.point_step = 12
    pc2_msg.row_step = pc2_msg.point_step * pc2_msg.width
    pc2_msg.is_dense = True

    buffer = []

    for point in points:
        float_bytes = [struct.pack('f', coord) for coord in point]
        buffer.append(b''.join(float_bytes))

    pc2_msg.data = b''.join(buffer)

    return pc2_msg

In [ ]:
def numpy_to_pointcloud(points, frame_id="base_link"):
    pc_msg = PointCloud()
    pc_msg.header.stamp = rospy.Time.now()
    pc_msg.header.frame_id = frame_id

    for point in points:
        p = Point32()
        p.x = point[0]
        p.y = point[1]
        p.z = point[2]
        pc_msg.points.append(p)

    return pc_msg

In [ ]:
shelf_mesh = trimesh.load_mesh('constraint_scene/Gruppe_21/model.stl')
shelf_mesh.apply_scale((0.02, 0.03, 0.02))
rot_mat = quaternion_to_rotation_matrix([shelf_pose.pose.orientation.w, shelf_pose.pose.orientation.x, shelf_pose.pose.orientation.y, shelf_pose.pose.orientation.z])
shelf_mesh.apply_transform(rot_mat)
shelf_mesh.apply_translation((shelf_pose.pose.position.x, shelf_pose.pose.position.y, shelf_pose.pose.position.z))
# generate pointcloud
obstacle_point_cloud = sample_points_on_mesh(shelf_mesh, 2000)

In [ ]:
pointcloud_pub = rospy.Publisher("/obstacle_point_cloud", PointCloud2, queue_size=1)

In [ ]:
point_cloud_msg = numpy_to_pointcloud2(obstacle_point_cloud, frame_id="base_link")
pointcloud_pub.publish(point_cloud_msg)

In [ ]:
horizontal_constraint = Constraints()
horizontal_constraint.name = "use_equality_constraints"

oc = OrientationConstraint()

oc.parameterization = OrientationConstraint.ROTATION_VECTOR;
oc.header.frame_id = "base_link";
oc.header.stamp = rospy.Time(0)
oc.link_name = "wrist_roll_link";
constrained_quaternion = Quaternion();
constrained_quaternion.x = 0.0
constrained_quaternion.y = 0.0
constrained_quaternion.z = 0.0
constrained_quaternion.w = 1.0
oc.orientation = constrained_quaternion
oc.weight = 1.0

oc.absolute_x_axis_tolerance = 0.1
oc.absolute_y_axis_tolerance = 0.1
oc.absolute_z_axis_tolerance = 2 * 3.1415
horizontal_constraint.orientation_constraints.append(oc)

# need to set in-hand pose
in_hand_pose = Pose()
in_hand_pose.position.x = 0.0
in_hand_pose.position.y = 0.0
in_hand_pose.position.z = 0.0
in_hand_pose.orientation.x = 0.0
in_hand_pose.orientation.y = 0.0
in_hand_pose.orientation.z = 0.0
in_hand_pose.orientation.w = 1.0
horizontal_constraint.in_hand_pose = in_hand_pose

In [ ]:
sample_request = GetJointWithConstraintsRequest()
sample_request.constraints = horizontal_constraint
sample_request.group_name = "arm"
init_sample = sample_joint_with_constraints_service(sample_request)
goal_sample = sample_joint_with_constraints_service(sample_request)
print("error code of init sample ", init_sample.error_code)
print("error code of target sample ", goal_sample.error_code)

In [ ]:
def getProperJointState(all_name, all_joint, selected_name):
    result = []
    for sn in selected_name:
        result.append(all_joint[all_name.index(sn)])
    return result

In [ ]:
init_joint = getProperJointState(init_sample.solution.joint_state.name, init_sample.solution.joint_state.position, joint_names)
goal_joint = getProperJointState(goal_sample.solution.joint_state.name, goal_sample.solution.joint_state.position, joint_names)

In [ ]:
print init_joint
print goal_joint
goal_joint = [-0.08726646259971647, 0.6806784082777885, 1.2566370614359172, -2.1467549799530254, -2.426007660272118, -1.9024088846738192, 2.181661564992912]

In [ ]:
moveit_robot_state = RobotState()
moveit_robot_state.joint_state.name = joint_names
moveit_robot_state.joint_state.position = init_joint

# move_group.set_planner_id('CBIRRTConfigDefault')
move_group.set_planner_id('CVQMPTRRTConfigDefault')
move_group.set_planning_time(10.0)
move_group.set_path_constraints(horizontal_constraint)
move_group.set_in_hand_pose(in_hand_pose)
move_group.set_clean_planning_context_flag(True)
move_group.set_obstacle_point_cloud(numpy_to_pointcloud(obstacle_point_cloud))

move_group.set_start_state(moveit_robot_state)
move_group.set_joint_value_target(goal_joint)
result = move_group.plan()

In [ ]:
move_group.set_planner_id('CBIRRTConfigDefault')
# move_group.set_planner_id('CVQMPTRRTConfigDefault')
move_group.set_planning_time(10.0)
move_group.set_path_constraints(horizontal_constraint)
move_group.set_in_hand_pose(in_hand_pose)
move_group.set_clean_planning_context_flag(True)
move_group.set_obstacle_point_cloud(numpy_to_pointcloud(obstacle_point_cloud))

move_group.set_start_state(moveit_robot_state)
move_group.set_joint_value_target(goal_joint)
result = move_group.plan()

In [ ]:
import math
degrees = [-5,39,72,-123,-139,-109,125]
radians = [math.radians(d) for d in degrees]

print(radians)